In [ ]:
from importlib import reload
import os
import numpy as np
import time
import glob
from PIL import Image
from tqdm import trange
from pulsestreamer import *
from nidaqmx import *

import analysis
reload(analysis)
from analysis import *

import measurement
reload(measurement)
from measurement import *

In [ ]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        # 'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        # 'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [ ]:
laser_array = np.linspace(1,26,2000)
steps = 18
laser_yarray = np.zeros((steps,laser_array.shape[0]))
trigger_yarray = np.zeros((steps,laser_array.shape[0]))
timing_yarray = np.zeros((steps,laser_array.shape[0]))
samples_xpoints = np.zeros((steps))
left_upper_samples_ypoints = np.zeros((steps,laser_array.shape[0]))
left_lower_samples_ypoints = np.zeros((steps,laser_array.shape[0]))
right_upper_samples_ypoints = np.zeros((steps,laser_array.shape[0]))
right_lower_samples_ypoints = np.zeros((steps,laser_array.shape[0]))


for k in range(0,steps,1):
    for j,i in enumerate(laser_array):
        allowed_steps = int(laser_array[-1]-7)
        if k>=allowed_steps:
            print('reduce the number of steps to :', allowed_steps)
            break
        if i<=2+1 or (i>=3+1 and i<=4+1) or (i>=5+1 and i<=6+1+k) or i>=7+1+k :
            laser_yarray[k,j]=0+1
            samples_xpoints[k]=7.5+1+k
        else:
            laser_yarray[k,j]=1+1
            
        if i>6.5+1+k and i<=7.5+1+k:
            left_upper_samples_ypoints[k,j]=7
            left_lower_samples_ypoints[k,j]=0
        else:
            left_upper_samples_ypoints[k,j]=None
            left_lower_samples_ypoints[k,j]=None
            
                    
        if i>=1.5 and i<2.5:
            right_upper_samples_ypoints[k,j]=7
            right_lower_samples_ypoints[k,j]=0
        else:
            right_upper_samples_ypoints[k,j]=None
            right_lower_samples_ypoints[k,j]=None
    
            
for k in range(0,steps,1):
    for j,i in enumerate(laser_array):
        allowed_steps = int(laser_array[-1]-7)
        if k>=allowed_steps:
            print('reduce the number of steps to :', allowed_steps)
            break           
        if i<=2+1 or (i>=2.7+1 and i<=4+1) or (i>=4.7+1 and i<=6+1+k) or i>=6.7+1+k :
            trigger_yarray[k,j]=2+1
        else:
            trigger_yarray[k,j]=3+1
    
            
for k in range(0,steps,1):
    for j,i in enumerate(laser_array):
        allowed_steps = int(laser_array[-1]-7)
        if k>=allowed_steps:
            print('reduce the number of steps to :', allowed_steps)
            break 
        if i<=2+1 or (i>=2.5+1 and i<=4+1) or (i>=4.5+1 and i<=6+1+k) or i>=6.5+1+k :
            timing_yarray[k,j]=4+1
        else:
            timing_yarray[k,j]=5+1

In [ ]:
for k in range(0,steps,1):
    y_samples = np.linspace(0,7,100)
    fig1 = simple_plot(x=laser_array,y=laser_yarray[k],show=False,curve_name='laser_pulse',mode='lines',width=800,height=600)
    fig2 = simple_plot(x=laser_array,y=trigger_yarray[k],show=False,curve_name='trigger_pulse',mode='lines',title=date,width=800,height=600)
    fig3 = simple_plot(x=laser_array,y=timing_yarray[k],show=False,curve_name='timing_pulse',mode='lines',width=800,height=600)
    fig4 = simple_plot(x=np.ones((y_samples.shape[0]))*samples_xpoints[k],y=y_samples,show=False,curve_name='',mode='lines',width=800,height=600,showlegend=False)
    fig5 = simple_plot(x=np.ones((y_samples.shape[0]))*1.5,y=y_samples,show=False,curve_name='',mode='lines',width=800,height=600,showlegend=False)
    fig6 = simple_plot(x=laser_array,y=left_upper_samples_ypoints[k],show=False,curve_name='',mode='lines',width=800,height=600,showlegend=False)
    fig7 = simple_plot(x=laser_array,y=left_lower_samples_ypoints[k],show=False,curve_name='',mode='lines',width=800,height=600,showlegend=False)
    fig8 = simple_plot(x=laser_array,y=right_upper_samples_ypoints[k],show=False,curve_name='',mode='lines',width=800,height=600,showlegend=False)
    fig9 = simple_plot(x=laser_array,y=right_lower_samples_ypoints[k],show=False,curve_name='',mode='lines',width=800,height=600,showlegend=False)
    
    fig4.line.color = "grey"
    fig5.line.color = "grey"
    fig6.line.color = "grey"
    fig7.line.color = "grey"
    fig8.line.color = "grey"
    fig9.line.color = "grey"

    name = f'steps_{k}.png'
    add_figures([fig1,fig2,fig3,fig4,fig5,fig6,fig7,fig8,fig9],show=True,title='pulse sequence (T<sub>1</sub> lifetime)',x_name = "Time (arb. u.)",y_name = "voltage (3.5 V)")

In [ ]:
tri_array = np.linspace(1,25,100)
sing_array = np.linspace(26,50,100)
steps = 18

tri_gd_0 = 1*np.ones((tri_array.shape[0])); tri_gd_1 = 3*np.ones((tri_array.shape[0]))
tri_ex_0 = 6*np.ones((tri_array.shape[0])); tri_ex_1 = 7*np.ones((tri_array.shape[0]))
sing_gd = 4*np.ones((sing_array.shape[0])); sing_ex = 5*np.ones((sing_array.shape[0]))

In [ ]:
import plotly.express as px

for i in range(steps):   
    fig=go.Figure()
    fig.add_scatter(x=tri_array[:int(tri_array.shape[0]/2)-14],y=tri_gd_0[:int(tri_array.shape[0]/2)-14],name='m<sub>s</sub>=0',mode='lines',line=dict(color='grey'))
    fig.add_scatter(x=tri_array[int(tri_array.shape[0]/2)+6:],y=tri_gd_0[int(tri_array.shape[0]/2)+6:],name='m<sub>s</sub>=0',mode='lines',line=dict(color='grey'))
    fig.add_scatter(x=tri_array[:int(tri_array.shape[0]/2)-14],y=tri_gd_1[:int(tri_array.shape[0]/2)-14],name='m<sub>s</sub>=0',mode='lines',line=dict(color='grey'))
    fig.add_scatter(x=tri_array[int(tri_array.shape[0]/2)+6:],y=tri_gd_1[int(tri_array.shape[0]/2)+6:],name='m<sub>s</sub>=0',mode='lines',line=dict(color='grey'))
    fig.add_scatter(x=tri_array,y=tri_ex_0,name='m<sub>s</sub>=0',mode='lines',line=dict(color='grey'))
    fig.add_scatter(x=tri_array,y=tri_ex_1,name='m<sub>s</sub>=&#177;1',mode='lines',line=dict(color='grey'))
    fig.add_scatter(x=sing_array,y=sing_gd,name='<sup>1</sup>A',mode='lines',line=dict(color='grey'))
    fig.add_scatter(x=sing_array,y=sing_ex,name='<sup>1</sup>E',mode='lines',line=dict(color='grey'))

    fig.update_layout(template = fig_template,width=700,height=600,showlegend=False,
                     title='equilibration process (T<sub>1</sub> lifetime)',
                     ) 

    fig.update_layout(
        xaxis=dict(
            showline=False,
            showticklabels=False, 
            tickwidth=0
        ),
        yaxis=dict(
            showline=False,
            showticklabels=False,
            tickwidth=0
        )
    )

    radius = 0.5
    num_points = 100  

    theta = np.linspace(0, 2 * np.pi, num_points)
    x = (radius+2) * np.cos(theta)+int(tri_array[-1]/2)
    y = (radius-0.12) * np.sin(theta)+tri_gd_0[-1]

    loweropacity = 1.0-0.5*(i/17)
    upperopacity = 0.1+0.4*(i/17)
    lowerfillcolor = f'rgba(255,127,80, {loweropacity})'
    upperfillcolor = f'rgba(255,127,80, {upperopacity})'
    fig.add_trace(go.Scatter(x=x, y=y,fill='toself', fillcolor=lowerfillcolor,
                             mode='lines', name='Circle',line=dict(color='grey')))
    fig.add_trace(go.Scatter(x=x, y=y+2, fill='toself',fillcolor=upperfillcolor,
                             mode='lines', name='Circle',line=dict(color='grey')))

    fig.add_annotation(
        x=int(tri_array[-1]/2), y=tri_gd_1[-1]-radius-0.5,
        ax=int(tri_array[-1]/2), ay=tri_gd_0[-1]+0.2,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=1.0,
        arrowwidth=4,
        arrowcolor=lowerfillcolor,
    )
    fig.add_annotation(
        x=int(tri_array[-1]/2), y=tri_gd_0[-1]+1.58,
        ax=int(tri_array[-1]/2), ay=tri_gd_1[-1]-radius-0.73,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=1.0,
        arrowwidth=4,
        arrowcolor=upperfillcolor,
    )

    fig.add_annotation(
        x=-3, y=1,
        ax=-6, ay=1,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='m<sub>s</sub> = 0'
    )
    fig.add_annotation(
        x=-3, y=3,
        ax=-6, ay=3,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='m<sub>s</sub> = &plusmn;1'
    )
    fig.add_annotation(
        x=-3, y=6,
        ax=-6, ay=6,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='m<sub>s</sub> = 0'
    )
    fig.add_annotation(
        x=-3, y=7,
        ax=-6, ay=7,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='m<sub>s</sub> = &plusmn;1'
    )
    fig.add_annotation(
        x=-10, y=2,
        ax=-7, ay=2,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='<sup>3</sup>A'
    )
    fig.add_annotation(
        x=-10, y=6.5,
        ax=-7, ay=6.5,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='<sup>3</sup>E'
    )


    fig.add_annotation(
        x=50+2, y=4,
        ax=53+2, ay=4,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='<sup>1</sup>E'
    )
    fig.add_annotation(
        x=50+2, y=5,
        ax=53+2, ay=5,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='<sup>1</sup>A'
    )
    fig.add_annotation(
        x=16+2, y=2,
        ax=30+2, ay=2,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text='phonon'
    )
    fig.add_annotation(
        x=25, y=8,
        ax=25, ay=8,
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=False,
        text=f'step : {i}',
        font=dict(
            family="Times new roman",
            size=25,
            color="orange"
        )
    )
    imagename = 
    fig.write_image(image)
    fig.show()

In [ ]:
import glob
from PIL import Image

path = os.path.join(os.getcwd(),'process_images')
def make_gif(frame_folder):
    frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
    frame_one = frames[0]
    frame_one.save(path + "/T1_process.gif", format="GIF", append_images=frames,
               save_all=True, duration=250, loop=0)
    

if __name__ == "__main__":
    make_gif(path)

In [ ]:
import glob
from PIL import Image

path = os.path.join(os.getcwd(),'seqeunce_images')
def make_gif(frame_folder):
    frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
    frame_one = frames[0]
    frame_one.save(path + "/T1_sequence.gif", format="GIF", append_images=frames,
               save_all=True, duration=250, loop=0)
    

if __name__ == "__main__":
    make_gif(path)